In [103]:
import os
import numpy as np
import shutil
from scipy import sparse

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.contrib.layers import apply_regularization, l2_regularizer
import bottleneck as bn

In [104]:
import vaecf_util as util
import vaecf as model
import vaecf_metric as metric
import vaecf_preprocess as preprocess
import vaecf_train as trainer
import vaecf_evaluate as evaluator

In [105]:
import imp
util = imp.reload(util)
preprocess = imp.reload(preprocess)
trainer = imp.reload(trainer)
evaluator = imp.reload(evaluator)
metric = imp.reload(metric)

In [106]:
### Pre-process data

In [107]:
DATA_DIR = '../data/'
fn_uid_sids = 'uid_sids.txt'

In [108]:
idx2uid, uid2idx, idx2sid, sid2idx, uid_sids_dict, n_users, n_items, train_data, vad_data_tr, vad_data_te, test_data_tr, test_data_te =\
    preprocess.preprocess_data(data_dir=DATA_DIR, fn_uid_sids=fn_uid_sids)

filter out items watched by and users watches less than:5
../data/data_train.txt ../data/data_dev.txt ../data/data_test.txt
n_users:15932
n_items:10401


In [109]:
### train
trainer = imp.reload(trainer)
metric = imp.reload(metric)

In [110]:
vae = trainer.train(n_users, n_items, train_data, vad_data_tr, vad_data_te, test_data_tr, test_data_te)

INFO:tensorflow:Scale of 0 disables regularizer.
log directory: ./log/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I
chkpt directory: ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I
epoch:0
  batch_num:0 start_index:0 end_index:5000 N:12746
  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 end_index:1593 N_vad:1593
  cur_ndcg:0.07546610010623789 best_ndcg:-inf
epoch:1
  batch_num:0 start_index:0 end_index:5000 N:12746
  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 end_index:1593 N_vad:1593
  cur_ndcg:0.10459033115842746 best_ndcg:0.07546610010623789
epoch:2
  batch_num:0 start_index:0 end_index:5000 N:12746
  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 end_index:1593 N_vad:1593
  cur_ndcg:0.0943468478548256 best_ndc

  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 end_index:1593 N_vad:1593
  cur_ndcg:0.261394411122332 best_ndcg:0.25352132543518385
epoch:29
  batch_num:0 start_index:0 end_index:5000 N:12746
  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 end_index:1593 N_vad:1593
  cur_ndcg:0.26769897057785735 best_ndcg:0.261394411122332
epoch:30
  batch_num:0 start_index:0 end_index:5000 N:12746
  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 end_index:1593 N_vad:1593
  cur_ndcg:0.2749785021248301 best_ndcg:0.26769897057785735
epoch:31
  batch_num:0 start_index:0 end_index:5000 N:12746
  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 end_inde

  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 end_index:1593 N_vad:1593
  cur_ndcg:0.39308247721908723 best_ndcg:0.3915214569346047
epoch:58
  batch_num:0 start_index:0 end_index:5000 N:12746
  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 end_index:1593 N_vad:1593
  cur_ndcg:0.39516370102499127 best_ndcg:0.39308247721908723
epoch:59
  batch_num:0 start_index:0 end_index:5000 N:12746
  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 end_index:1593 N_vad:1593
  cur_ndcg:0.39822170674331575 best_ndcg:0.39516370102499127
epoch:60
  batch_num:0 start_index:0 end_index:5000 N:12746
  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 end_

  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 end_index:1593 N_vad:1593
  cur_ndcg:0.4263528453883466 best_ndcg:0.4244503074083598
epoch:87
  batch_num:0 start_index:0 end_index:5000 N:12746
  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 end_index:1593 N_vad:1593
  cur_ndcg:0.4266199672628435 best_ndcg:0.4263528453883466
epoch:88
  batch_num:0 start_index:0 end_index:5000 N:12746
  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 end_index:1593 N_vad:1593
  cur_ndcg:0.42629189096287584 best_ndcg:0.4266199672628435
epoch:89
  batch_num:0 start_index:0 end_index:5000 N:12746
  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 end_inde

  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 end_index:1593 N_vad:1593
  cur_ndcg:0.43037341186545625 best_ndcg:0.4332392345349872
epoch:116
  batch_num:0 start_index:0 end_index:5000 N:12746
  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 end_index:1593 N_vad:1593
  cur_ndcg:0.43149280492776515 best_ndcg:0.4332392345349872
epoch:117
  batch_num:0 start_index:0 end_index:5000 N:12746
  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 end_index:1593 N_vad:1593
  cur_ndcg:0.4337743530003252 best_ndcg:0.4332392345349872
epoch:118
  batch_num:0 start_index:0 end_index:5000 N:12746
  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 end_

  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 end_index:1593 N_vad:1593
  cur_ndcg:0.43448771793977964 best_ndcg:0.43632273714204023
epoch:145
  batch_num:0 start_index:0 end_index:5000 N:12746
  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 end_index:1593 N_vad:1593
  cur_ndcg:0.4338773778748214 best_ndcg:0.43632273714204023
epoch:146
  batch_num:0 start_index:0 end_index:5000 N:12746
  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 end_index:1593 N_vad:1593
  cur_ndcg:0.4326034261672515 best_ndcg:0.43632273714204023
epoch:147
  batch_num:0 start_index:0 end_index:5000 N:12746
  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 en

  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 end_index:1593 N_vad:1593
  cur_ndcg:0.4370879275431487 best_ndcg:0.4379086456960479
epoch:174
  batch_num:0 start_index:0 end_index:5000 N:12746
  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 end_index:1593 N_vad:1593
  cur_ndcg:0.43894939580672054 best_ndcg:0.4379086456960479
epoch:175
  batch_num:0 start_index:0 end_index:5000 N:12746
  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 end_index:1593 N_vad:1593
  cur_ndcg:0.43894570625763 best_ndcg:0.43894939580672054
epoch:176
  batch_num:0 start_index:0 end_index:5000 N:12746
  batch_num:1 start_index:5000 end_index:10000 N:12746
  batch_num:2 start_index:10000 end_index:12746 N:12746
  batch_num:0 start_index:0 end_in

In [111]:
### evaluate

In [116]:
evaluator.evaluate(n_users, n_items, train_data, vad_data_tr, vad_data_te, test_data_tr, test_data_te)

INFO:tensorflow:Scale of 0 disables regularizer.
chkpt directory: ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I
INFO:tensorflow:Restoring parameters from ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I/model
Test NDCG@100=0.41498 (0.00802)
Test Recall@20=0.45866 (0.00937)
Test Recall@50=0.54941 (0.00931)


In [119]:
### predict
evaluator = imp.reload(evaluator)

In [149]:
base_sididx_arr = None
uid = '66049978'
if uid not in uid2idx:
    print('uid not in uid2idx')
else:
    raw_sids = uid_sids_dict[uid]
    sids = []
    for sid in raw_sids:
        if sid in sid2idx:
            sids.append(sid)
    base_sid_arr = sids
    base_sididx_arr = [sid2idx[sid] for sid in base_sid_arr]
print(base_sid_arr)

['M984501312', 'M994244556', 'M1001219330', 'M1312793527', 'U57738229384', 'M1023606622', 'M984421851']


In [150]:
scores = evaluator.predict(base_sididx_arr, n_items)

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from ./chkpt/VAE_anneal200.0K_cap2.0E-01/I-600-200-600-I/model


In [151]:
scores = scores.ravel()
topk = 20
indices = np.argpartition(-scores, topk)[:topk]
print([idx2sid[sididx] for sididx in indices])
print(scores[indices][:topk])

['M997120535', 'M1025707874', 'U61124273589', 'M1029095653', 'M1000802966', 'M1096598870', 'M1019879977', 'M1032856732', 'M998144258', 'M1023707444', 'M1213884323', 'M1322432451', 'M1121915816', 'S4720515124993427383', 'M1005962014', 'M1016280242', 'M1032230840', 'M1020653513', 'M1018572263', 'M1016250500']
[8.553545  9.118637  7.150369  7.291399  7.8064413 7.210292  7.834192
 7.404824  7.910143  7.867538  7.3831906 7.1366043 7.4158087 7.0833893
 8.520889  8.123861  7.5788264 7.301521  8.621164  8.143372 ]
